In [ ]:
import pandas as pd
import numpy as np
import warnings
import itertools
import plotly.plotly as ply
from math import sqrt
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("House1_1min_Filled.csv",index_col = 0)
df.head()
df.index = pd.to_datetime(df.index)
df = df.resample("H", label='right', closed='right').sum()
df.head()

In [ ]:
df = df.iloc[:,:7]
df.head()

In [ ]:
household = pd.DataFrame(df.sum(axis=1),columns = ["Total Electricity"] )
household

In [ ]:
X_1 = household["Total Electricity"].values
size = int(len(X_1) * 0.75)
train, test = X_1[0:size], X_1[size:len(X_1)]

In [ ]:
from pandas import datetime
from pandas import DataFrame
from pandas import concat
 
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df
 
# transform to supervised learning
supervised = timeseries_to_supervised(X_1, 1)
print(supervised.head())

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)
 

In [ ]:
# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [ ]:
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [ ]:
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [ ]:
# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model

In [ ]:
# make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# transform data to be supervised learning
supervised_values = supervised.values
 
# split data into train and test-sets
size = int(len(supervised_values) * 0.75)
train, test = supervised_values[0:size], supervised_values[size:len(supervised_values)]
 
# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)


In [ ]:
# fit the model
lstm_model = fit_lstm(train_scaled, 1,500, 1)

In [ ]:
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

In [ ]:
# walk-forward validation on the test data
predictions = list()
for i in range(len(test_scaled)):
    # make one-step forecast
    X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
    yhat = forecast_lstm(lstm_model, 1, X)
    # invert scaling
    yhat = invert_scale(scaler, X, yhat)
    # invert differencing
    yhat = inverse_difference(X_1, yhat, len(test_scaled)+1-i)
    # store forecast
    predictions.append(yhat)
    expected = X_1[len(train) + i + 1]
    print('Time=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

In [ ]:
# report performance
rmse = sqrt(mean_squared_error(X_1[size:len(X_1)], predictions))
print('Test RMSE: %.3f' % rmse)
# line plot of observed vs predicted
pyplot.plot(X_1[size:len(X_1)])
pyplot.plot(predictions)
pyplot.show()

In [ ]:
from plotly import __version__
%matplotlib inline
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)

init_notebook_mode(connected=True)
cf.go_offline()


In [ ]:
# plot the actual consumption and the prediction 
future_forecast = pd.DataFrame(predictions,index = household["Total Electricity"][int(len(household.values) * 0.75):].index,columns=['Prediction'])
pd.concat([household["Total Electricity"],future_forecast],axis=1).iplot()

In [ ]:
# To prevent lstm from overfitting
repeats = 30
error_scores = list()
for r in range(repeats):
    # fit the model
    lstm_model = fit_lstm(train_scaled, 1, 500, 1)
    # forecast the entire training dataset to build up state for forecasting
    train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
    lstm_model.predict(train_reshaped, batch_size=1)
    # walk-forward validation on the test data
    predictions = list()
    for i in range(len(test_scaled)):
        # make one-step forecast
        X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
        yhat = forecast_lstm(lstm_model, 1, X)
        # invert scaling
        yhat = invert_scale(scaler, X, yhat)
        # invert differencing
        yhat = inverse_difference(X_1, yhat, len(test_scaled)+1-i)
        # store forecast
        predictions.append(yhat)
    # report performance
    rmse = sqrt(mean_squared_error(X_1[-12:], predictions))
    print('%d) Test RMSE: %.3f' % (r+1, rmse))
    error_scores.append(rmse)
